In [2]:
%matplotlib inline
from ipywidgets import interactive, FloatSlider
import matplotlib.pyplot as plt
import numpy as np
import os
from astropy.io import fits
import warnings

# Filter out runtime warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

def loading_FITS(string_filepath, extension=0):
    try:
        # Check if the filepath exists
        if not os.path.exists(string_filepath):
            raise FileNotFoundError("File does not exist")
        # Try entering a number instead of a string
        # for isinstance: https://www.w3schools.com/python/ref_func_isinstance.asp
        assert isinstance(string_filepath, str), 'Filepath must be a string'      
        # Define valid extensions
        # endswith python: https://www.w3schools.com/python/ref_string_endswith.asp
        endings = ['.FIT', '.FITS', '.fit', '.fits']
        # Check if the ending of string_filepath matches any valid extension
        assert any(string_filepath.endswith(i) for i in endings), "Must be of type .FIT, .FITS, .fit, .fits"
        with fits.open(string_filepath) as hdu:
            # check if the extension is in range or not
            extension_total = len(hdu)
            assert extension < extension_total, 'Extension not in range, the extension value must be less than {}'.format(extension_total)
            header = hdu[extension].header
            data = hdu[extension].data
        return header, data
    except (AssertionError, ValueError, FileNotFoundError) as msg:
        print(msg)
        return None, None

def plot_image(data, vmin, vmax, figsize=(5, 5), cmap='gray_r', x_axis='Default X Coordinates', y_axis='Default Y Coordinates', title='Title'):
    if data is not None:
        data_log = np.where(data > 0, np.log(data), 0)
        fig, ax = plt.subplots(figsize=figsize)
        # still hardcoding the rotation 
        data_rotated = np.rot90(data_log, 2) 
        data_rotated[data_rotated == 0] = np.nan
        # Plotting and setting titles
        im = ax.imshow(data_rotated, cmap=cmap, vmin=vmin, vmax=vmax)
        title = f'Log Scale vmin={vmin}, vmax={vmax}'
        ax.set_title(title)
        ax.set_xlabel(x_axis)
        ax.set_ylabel(y_axis)
        plt.colorbar(im, ax=ax)
        plt.show()

header, data = loading_FITS('M51_HST (2).fits')

# Define sliders for vmin and vmax
vmin_slider = FloatSlider(min=np.min(data)-10, max=np.max(data), step=0.1, description='vmin', value=np.min(data))
vmax_slider = FloatSlider(min=np.min(data), max=np.max(data)+10, step=0.1, description='vmax', value=np.max(data))
# Reduce Resolution of the image to ten times less of the original to help with speed or processing 
subsample_slider = FloatSlider(min=1, max=10, step=1, description='Subsample Factor', value=10)

# Define interactive plot function
def interactive_plot(vmin, vmax):
    plot_image(data, vmin, vmax)

# Create interactive plot
interactive_plot = interactive(interactive_plot, vmin=vmin_slider, vmax=vmax_slider, subsample_factor=subsample_slider)
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

# use arrow pad to make it easier 

interactive(children=(FloatSlider(value=-0.6049584746360779, description='vmin', max=274.4317932128906, min=-1…